In [28]:
# %pip install wikipedia==1.4.0 google-search-results==2.4.2 better-profanity==0.7.0
#%pip install langchain
# #install transformers
# %pip install transformers
#%pip install langchain-community
#%pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
#%pip install OpenAI





#%pip install better_profanity
# %pip install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu118
# %pip install -U langgraph langchain-openai

   ---------------------------------------- 0.0/883.8 kB ? eta -:--:--
   ---------------------------------------- 883.8/883.8 kB 6.7 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [5]:
#test torch installation
# import torch
# x = torch.rand(5, 3)
# print(x)

In [40]:
#generate API tokens
import os
import json

file_path = r"C:\Users\lamke\Documents\AI web crash course\LLM\large-language-models\credentials.json"

with open(file_path, 'r') as f:
     credentials = json.load(f)
os.environ["HUGGINGFACEHUB_API_TOKEN"] = credentials["huggingface_key"]
os.environ["SERPAPI_API_KEY"] = credentials["serpapi_api_key"]
os.environ["OPENAI_API_KEY"] = credentials["openai_key"]


In [45]:
from serpapi import GoogleSearch
# search = GoogleSearch({
#     "q": "coffee", 
#     "location": "Austin,Texas",
#     "api_key": credentials["serpapi_api_key"]
#   })
# result = search.get_dict()

In [47]:
import wikipedia
print(wikipedia.summary("Wikipedia"))

The Wikimedia Foundation, Inc., abbreviated WMF, is an American 501(c)(3) nonprofit organization headquartered in San Francisco, California, and registered there as a charitable foundation. It is the host of Wikipedia, the seventh most visited website in the world. It also hosts fourteen related open collaboration projects, and supports the development of MediaWiki, the wiki software which underpins them all. The foundation was established in 2003 in St. Petersburg, Florida by Jimmy Wales, as a non-profit way to fund Wikipedia and other wiki projects which had previously been hosted by Bomis, Wales' for-profit company.
The Wikimedia Foundation provides the technical and organizational infrastructure to enable members of the public to develop wiki-based content in languages across the world. The foundation does not write or curate any of the content on the projects themselves. Instead, this is done by volunteer editors, such as the Wikipedians. However, it does collaborate with a networ

In [7]:
#JekyllHyde
#prompt template
from langchain import PromptTemplate
import numpy as np

#this is the template for Jekyll to tell it how to response, and what variables to use
jekyll_template = """
You are a social media post commenter, you will respond to the following post with a {sentiment} response. 
Post:" {social_post}"
Comment: 
"""

#use this class to create an instance to use the prompt from above and store variables we need
jekyll_prompt_template = PromptTemplate(
    input_variables=["sentiment", "social_post"],
    template=jekyll_template,
)

#include some randomized sentiment
random_sentiment = "nice"
if np.random.rand() < 0.3:
    random_sentiment = "mean"

social_post = "I can't believe I'm learning about LangChain in this MOOC, there is so much to learn and so far the instructors have been so helpful. I'm having a lot of fun learning! #AI #Databricks"

jekyll_prompt = jekyll_prompt_template.format(
    sentiment=random_sentiment,
    social_post=social_post,
)
print(f"Jekyll  prompt:{jekyll_prompt}")

Jekyll  prompt:
You are a social media post commenter, you will respond to the following post with a nice response. 
Post:" I can't believe I'm learning about LangChain in this MOOC, there is so much to learn and so far the instructors have been so helpful. I'm having a lot of fun learning! #AI #Databricks"
Comment: 



In [ ]:


#give Jekyll a brain
#To interact with LLMs in LangChain we need the following modules loaded
# Generate API tokens
from langchain.llms import HuggingFacePipeline 
from langchain.llms import OpenAI 

# import sys
# sys.modules['langchain_community'] = sys.modules['langchain.llms']

from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
# jekyll_llm = OpenAI(model="text-babbage-001") 
# text-babbage-001 has been deprecated and using GPT-3.5-turbo requires a paid subscription


c:\Users\lamke\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:

# #HuggingFaceHub alternative
#install transformers
model_id = "EleutherAI/gpt-neo-125m" # A smaller alternative model
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id)
pipe = pipeline('text-generation', model=model, tokenizer=tokenizer, max_new_tokens=512, device_map='auto')
jekyll_llm = HuggingFacePipeline(pipeline=pipe)

Device set to use cpu
C:\Users\lamke\AppData\Local\Temp\ipykernel_16172\2387260682.py:7: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  jekyll_llm = HuggingFacePipeline(pipeline=pipe)


In [10]:
#Building our prompt-LLM Chain
from langchain.chains import LLMChain
from better_profanity import profanity

jekyll_chain = LLMChain(
    prompt=jekyll_prompt_template,
    llm=jekyll_llm,
    output_key ="jekyll_said",
    verbose=True,
)  # Now that we've chained the LLM and prompt, the output of the formatted prompt will pass directly to the LLM.

#.run method will be used and variables are input as dict
jekyll_said = jekyll_chain.run(
    {"sentiment": random_sentiment, "social_post": social_post}
)

#before printing, clean it up
cleaned_jekyll_said = profanity.censor(jekyll_said)
print(f"Jekyll said: {cleaned_jekyll_said}")


C:\Users\lamke\AppData\Local\Temp\ipykernel_16172\375983532.py:5: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  jekyll_chain = LLMChain(
C:\Users\lamke\AppData\Local\Temp\ipykernel_16172\375983532.py:13: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  jekyll_said = jekyll_chain.run(
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.




> Entering new LLMChain chain...
Prompt after formatting:

You are a social media post commenter, you will respond to the following post with a nice response. 
Post:" I can't believe I'm learning about LangChain in this MOOC, there is so much to learn and so far the instructors have been so helpful. I'm having a lot of fun learning! #AI #Databricks"
Comment: 


> Finished chain.
Jekyll said: 
You are a social media post commenter, you will respond to the following post with a nice response. 
Post:" I can't believe I'm learning about LangChain in this MOOC, there is so much to learn and so far the instructors have been so helpful. I'm having a lot of fun learning! #AI #Databricks"
Comment: 

I'm not sure if you are using the same code as me, but I'm not sure if you are using the same code as me. 

A:

I think you are using the same code as me. 

I'm not sure if you are using the same code as me. 

I think you are using the same code as me. 

I think you are using the same code as me. 

In [11]:
#Build the second chain for Hyde moderator
hyde_template = """
You are Hyde, the moderator of an online forum, you are strict and will not tolerate any negative comments. 
We will look at this next comment from a user and, if it is at all negative, you will replace it with symbols and post that, but if it seems nice, you will let it remain as is and repeat it word by word
Original comment: {jekyll_said}
Edited comment: 
"""

##Use the PromptTemplate class to create instance and use the prompt from above and store variables we will need to input when making the prompt
hyde_prompt_template = PromptTemplate(
    input_variables=["jekyll_said"],
    template=hyde_template,
)

hyde_llm=jekyll_llm
#can use other model if needed

hyde_chain = LLMChain(
    llm=hyde_llm, prompt=hyde_prompt_template, verbose=False
)

hyde_says = hyde_chain.run({"jekyll_said": jekyll_said})
print(f"Hyde said: {hyde_says}")


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Hyde said: 
You are Hyde, the moderator of an online forum, you are strict and will not tolerate any negative comments. 
We will look at this next comment from a user and, if it is at all negative, you will replace it with symbols and post that, but if it seems nice, you will let it remain as is and repeat it word by word
Original comment: 
You are a social media post commenter, you will respond to the following post with a nice response. 
Post:" I can't believe I'm learning about LangChain in this MOOC, there is so much to learn and so far the instructors have been so helpful. I'm having a lot of fun learning! #AI #Databricks"
Comment: 

I'm not sure if you are using the same code as me, but I'm not sure if you are using the same code as me. 

A:

I think you are using the same code as me. 

I'm not sure if you are using the same code as me. 

I think you are using the same code as me. 

I think you are using the same code as me. 

I think you are using the same code as me. 

I think 

In [12]:
#combine the two chains and build a sequential chain = JekyllHyde

from langchain.chains import SequentialChain

#this class link the chains together. Input variables will also be added, can be at any point not just the start
jekyll_hyde_chain = SequentialChain(
    chains=[jekyll_chain, hyde_chain],
    input_variables=["sentiment", "social_post"],
    verbose=True,
)

#and now run the chain with randomized sentiment and social post
jekyll_hyde_output = jekyll_hyde_chain.run(
    {"sentiment": random_sentiment, "social_post": social_post}
)

print(f"JekyllHyde said: {jekyll_hyde_output}")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.




> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

You are a social media post commenter, you will respond to the following post with a nice response. 
Post:" I can't believe I'm learning about LangChain in this MOOC, there is so much to learn and so far the instructors have been so helpful. I'm having a lot of fun learning! #AI #Databricks"
Comment: 



Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



> Finished chain.

> Finished chain.
JekyllHyde said: 
You are Hyde, the moderator of an online forum, you are strict and will not tolerate any negative comments. 
We will look at this next comment from a user and, if it is at all negative, you will replace it with symbols and post that, but if it seems nice, you will let it remain as is and repeat it word by word
Original comment: 
You are a social media post commenter, you will respond to the following post with a nice response. 
Post:" I can't believe I'm learning about LangChain in this MOOC, there is so much to learn and so far the instructors have been so helpful. I'm having a lot of fun learning! #AI #Databricks"
Comment: 

I'm not sure if you are using the same code as me, but I'm not sure if you are using the same code as me. 

A:

I think you are using the same code as me. 

I'm not sure if you are using the same code as me. 

I think you are using the same code as me. 

I think you are using the same code as me. 

I think y

In [54]:
from langchain_core.messages import HumanMessage
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import create_react_agent
from serpapi import GoogleSearch

# Custom GoogleSearch tool class
class GoogleSearchTool:
    def __init__(self, api_key, max_results=2):
        self.api_key = api_key
        self.max_results = max_results

    def __call__(self, query):
        search = GoogleSearch({"q": query, "api_key": self.api_key})
        results = search.get_dict()
        # Extract top results
        if "organic_results" in results:
            return [
                result["title"] + " - " + result["link"]
                for result in results["organic_results"][: self.max_results]
            ]
        return "No results found."

# Create memory for the agent
memory = MemorySaver

# Initialize the LLM (assumes jekyll_llm is already defined)
model = jekyll_llm

# Set up the GoogleSearch tool
serp_api_key = "your_serpapi_api_key"  # Replace with your SerpAPI key
google_search_tool = GoogleSearchTool(api_key=serp_api_key)

# Wrap the tool in a callable format for LangChain
tools = [{"name": "google_search", "func": google_search_tool}]

# Create the ReAct agent
agent_executor = create_react_agent(model, tools, checkpointer=memory)

# Config and interaction
config = {"configurable": {"thread_id": "abc123"}}

# Interaction 1: Ask a question
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="Hi, I'm Bob! I live in SF.")]}, config
):
    print(chunk)
    print("----")

# Interaction 2: Query the weather in the user's location
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="What's the weather in San Francisco?")]}, config
):
    print(chunk)
    print("----")


ValueError: The first argument must be a string or a callable with a __name__ for tool decorator. Got <class 'dict'>